In [1]:
import os
#from pandas.tests.frame.methods.test_sort_values import ascending
from datetime import datetime, timezone, timedelta
os.environ["SPARK_HOME"] = "/mnt/d/learn/DE/Semina_project/spark"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,ArrayType
from pyspark.sql.functions import input_file_name, explode, col,lit, date_format, to_timestamp
import duckdb
import pyarrow as pa

In [12]:
S3_ENDPOINT = "http://localhost:9000"
S3_ACCESS = "minio"
S3_SECRET = "minio123"
S3_BUCKET = "trading-okx"
def get_latest_file(bucket_name, prefix,days_lookback):
    s3 = boto3.client(
        "s3",
        endpoint_url=S3_ENDPOINT,
        aws_access_key_id=S3_ACCESS,
        aws_secret_access_key=S3_SECRET,
        region_name="us-east-1"
    )
    all_objects = []
    # Paginator is used to list if more 1000 files
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    for page in page_iterator:
        for obj in page.get('Contents', []):
            key=obj['Key']
            last_modified = obj['LastModified']
            if last_modified > datetime.now(timezone.utc)- timedelta(days=days_lookback):
                 all_objects.append(obj)
    all_objects.sort(key=lambda x: x['LastModified'], reverse=True)
    #latest_files = all_objects[:limit]
    #latest_files = all_objects
    paths = [f"s3a://{bucket_name}/{obj['Key']}" for obj in all_objects]
    return paths

latest_files = get_latest_file(
        bucket_name=S3_BUCKET,
        prefix="bronze/okx_ohlc",
        days_lookback=1 # thay đổi khi run lại toàn bộ
)
latest_files

['s3a://trading-okx/bronze/okx_ohlc/2025-12-04/21/okx_ohlc_20251204_214557.jsonl.gz',
 's3a://trading-okx/bronze/okx_ohlc/2025-12-04/21/okx_ohlc_20251204_214553.jsonl.gz',
 's3a://trading-okx/bronze/okx_ohlc/2025-12-04/21/okx_ohlc_20251204_214554.jsonl.gz',
 's3a://trading-okx/bronze/okx_ohlc/2025-12-04/21/okx_ohlc_20251204_214552.jsonl.gz']

In [3]:
spark = SparkSession.builder \
    .appName("OKX_Bronze_To_Silver_Book_new") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
25/12/04 21:57:25 WARN Utils: Your hostname, hoalaptop resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/12/04 21:57:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/d/learn/DE/Semina_project/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hoalaptop/.ivy2/cache
The jars for the packages stored in: /home/hoalaptop/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0fffab50-0c84-4e89-bfea-052f4a3d9585;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 325ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------

In [13]:
candle_array_schema = ArrayType(StringType())
schema = StructType([
    StructField("received_at", StringType(), True),
    StructField("payload", StructType([
        # Lấy thêm thông tin channel để biết instId (Symbol)
        StructField("arg", StructType([
            StructField("instId", StringType(), True),
            StructField("channel", StringType(), True)
        ]), True),
        # Data là mảng của các mảng
        StructField("data", ArrayType(candle_array_schema), True)
    ]), True)
])

df = spark.read.schema(schema).json(latest_files)
df.show(truncate=False)

+--------------------------+-------------------------------------------------------------------------------------------------------------------+
|received_at               |payload                                                                                                            |
+--------------------------+-------------------------------------------------------------------------------------------------------------------+
|2025-12-04T21:36:03.390932|{{BTC-USDT-SWAP, candle1s}, [[1764858960000, 92570.9, 92597.9, 92560, 92580.3, 2165.9, 21.659, 2005248.35297, 1]]} |
|2025-12-04T21:36:08.509532|{{BTC-USDT-SWAP, candle1s}, [[1764858967000, 92550.7, 92552.9, 92542.9, 92542.9, 68.79, 0.6879, 63664.15765, 1]]}  |
|2025-12-04T21:36:25.655413|{{BTC-USDT-SWAP, candle1s}, [[1764859400000, 92504.1, 92504.1, 92504.1, 92504.1, 9.53, 0.0953, 8815.64073, 1]]}    |
|2025-12-04T21:43:33.237312|{{BTC-USDT-SWAP, candle1s}, [[1764859412000, 92585.8, 92593.1, 92585.8, 92593.1, 216.54, 2.1654, 20049

In [16]:
df_exploded = df.select(
    col("received_at"),
    col("payload.arg.instId").alias("symbol"),
    col("payload.arg.channel").alias("channel"),  # Ví dụ: candle1s, candle1m
    explode(col("payload.data")).alias("candle")  # Mỗi dòng là 1 mảng ["ts", "o", ...]
)
df_silver = df_exploded.select(
    col("symbol"),
    col("channel"),
    to_timestamp(col("candle")[0].cast("long") / 1000).alias("candle_time"),
    col("candle")[1].cast("double").alias("open"),
    col("candle")[2].cast("double").alias("high"),
    col("candle")[3].cast("double").alias("low"),
    col("candle")[4].cast("double").alias("close"),
    col("candle")[5].cast("double").alias("volume"),
    col("candle")[8].cast("int").alias("is_confirmed"),
    col("received_at").cast("timestamp").alias("ingestion_time")
)
df_silver.show()

+-------------+--------+-------------------+-------+-------+-------+-------+------+------------+--------------------+
|       symbol| channel|        candle_time|   open|   high|    low|  close|volume|is_confirmed|      ingestion_time|
+-------------+--------+-------------------+-------+-------+-------+-------+------+------------+--------------------+
|BTC-USDT-SWAP|candle1s|2025-12-04 21:36:00|92570.9|92597.9|92560.0|92580.3|2165.9|           1|2025-12-04 21:36:...|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:36:07|92550.7|92552.9|92542.9|92542.9| 68.79|           1|2025-12-04 21:36:...|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:43:20|92504.1|92504.1|92504.1|92504.1|  9.53|           1|2025-12-04 21:36:...|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:43:32|92585.8|92593.1|92585.8|92593.1|216.54|           1|2025-12-04 21:43:...|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:20:58|92580.0|92580.0|92580.0|92580.0|  3.68|           1|2025-12-04 21:21:...|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:40:52|92672.0|9267

In [18]:
#df.show(truncate=False)

#df_exploded.show(truncate=False)
# B3.2: Ép kiểu và Chọn cột

#df_silver.show(truncate=False)
df_cleaned = df_silver \
    .dropna(subset=["open", "candle_time"]) \
    .withColumn("date_part", date_format(col("candle_time"), "yyyy-MM-dd"))
df_cleaned.show()


+-------------+--------+-------------------+-------+-------+-------+-------+------+------------+--------------------+----------+
|       symbol| channel|        candle_time|   open|   high|    low|  close|volume|is_confirmed|      ingestion_time| date_part|
+-------------+--------+-------------------+-------+-------+-------+-------+------+------------+--------------------+----------+
|BTC-USDT-SWAP|candle1s|2025-12-04 21:36:00|92570.9|92597.9|92560.0|92580.3|2165.9|           1|2025-12-04 21:36:...|2025-12-04|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:36:07|92550.7|92552.9|92542.9|92542.9| 68.79|           1|2025-12-04 21:36:...|2025-12-04|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:43:20|92504.1|92504.1|92504.1|92504.1|  9.53|           1|2025-12-04 21:36:...|2025-12-04|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:43:32|92585.8|92593.1|92585.8|92593.1|216.54|           1|2025-12-04 21:43:...|2025-12-04|
|BTC-USDT-SWAP|candle1s|2025-12-04 21:20:58|92580.0|92580.0|92580.0|92580.0|  3.68|           1|2

In [20]:
df_cleaned.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- candle_time: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- is_confirmed: integer (nullable = true)
 |-- ingestion_time: timestamp (nullable = true)
 |-- date_part: string (nullable = true)



In [ ]:
output = f"s3a://trading-okx/silver/okx-funding/"
df_silver.write.mode('append').format("parquet").save(output)

# config dw duckdb

In [45]:
# connect to duckdb
duck_path='/mnt/d/learn/DE/Semina_project/datawarehouse.duckdb'
con=duckdb.connect(duck_path)

In [47]:
con.close()

In [35]:
# scrip create table
with open('/mnt/d/learn/DE/Semina_project/SQL_db/config_database/source_db.sql', 'r') as f:
    sql_script = f.read()
con.execute(sql_script)

In [46]:
# test table
con.sql("show TABLEs ; ")
#con.sql("select * from fact_ohlc;")


┌───────────────────┐
│       name        │
│      varchar      │
├───────────────────┤
│ fact_funding_rate │
│ fact_mark_price   │
│ fact_ohlc         │
│ fact_order_books  │
│ fact_trades       │
└───────────────────┘

In [ ]:
# read file sql to connect to minio and create dim fact table
with open('/mnt/d/learn/DE/Semina_project/SQL_db/config_dw/warehouse_source.sql', 'r') as f:
    sql_script = f.read()
con.execute(sql_script)

In [39]:
arrow_table = pa.Table.from_pandas(df_cleaned.toPandas())
con.register("arrow_table", arrow_table)
con.sql('select * from arrow_table')
#con.close()

┌───────────────┬──────────┬─────────────────────┬─────────┬─────────┬─────────┬─────────┬────────┬──────────────┬────────────────────────────┬────────────┐
│    symbol     │ channel  │     candle_time     │  open   │  high   │   low   │  close  │ volume │ is_confirmed │       ingestion_time       │ date_part  │
│    varchar    │ varchar  │    timestamp_ns     │ double  │ double  │ double  │ double  │ double │    int32     │        timestamp_ns        │  varchar   │
├───────────────┼──────────┼─────────────────────┼─────────┼─────────┼─────────┼─────────┼────────┼──────────────┼────────────────────────────┼────────────┤
│ BTC-USDT-SWAP │ candle1s │ 2025-12-04 21:36:00 │ 92570.9 │ 92597.9 │ 92560.0 │ 92580.3 │ 2165.9 │            1 │ 2025-12-04 21:36:03.390932 │ 2025-12-04 │
│ BTC-USDT-SWAP │ candle1s │ 2025-12-04 21:36:07 │ 92550.7 │ 92552.9 │ 92542.9 │ 92542.9 │  68.79 │            1 │ 2025-12-04 21:36:08.509532 │ 2025-12-04 │
│ BTC-USDT-SWAP │ candle1s │ 2025-12-04 21:43:20 │ 92504.1

In [40]:
con.execute('''
                INSERT INTO fact_ohlc(
                                            symbol,
                                            channel,
                                            candle_time,
                                            open,
                                            high,
                                            low,
                                            close,
                                            volume,
                                            is_confirmed,
                                            ingestion_time,
                                            date_part)
                SELECT
                    symbol ,
                    channel ,
                    candle_time ,
                    open ,
                    high ,
                    low ,
                    close ,
                    volume ,
                    is_confirmed ,
                    ingestion_time ,
                    date_part
                FROM arrow_table
                ''')

In [ ]:
# read file sql to connect to minio and create dim fact table
with open('/mnt/d/learn/DE/Semina_project/SQL_db/config_dw/warehouse_source.sql', 'r') as f:
    sql_script = f.read()
con.execute(sql_script)

output_path = f"s3://trading-okx/silver/okx-funding/*.parquet"
try:
    # Chỉ load dữ liệu mới (Ví dụ lọc theo ngày nếu cần)
    # Ở đây load hết (Full Load)
    con.execute(f"""
        INSERT INTO fact_funding_rate
        SELECT * FROM read_parquet('{output_path}')
       --WHERE ingestion_time NOT IN (SELECT ingestion_time FROM fact_orders_books) -- Tránh trùng lặp (Dedup đơn giản)
    """)

    # Lấy số dòng đã insert
    count = con.execute("SELECT count(*) FROM fact_funding_rate").fetchone()[0]
    print(f"✅ Data loaded. Total rows in DuckDB: {count}")

except Exception as e:
    print(f"❌ Error loading data: {e}")


In [ ]:
con.close()
spark.stop()

In [14]:
import boto3
import pyarrow as pa
from datetime import datetime, timezone, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,ArrayType
from pyspark.sql.functions import input_file_name, explode, col, date_format
import duckdb
''' code dùng cho fact table'''
S3_ENDPOINT = "http://localhost:9000"
S3_ACCESS = "minio"
S3_SECRET = "minio123"
S3_BUCKET = "trading-okx"
#
def get_latest_file(bucket_name, prefix,days_lookback):
    s3 = boto3.client(
        "s3",
        endpoint_url=S3_ENDPOINT,
        aws_access_key_id=S3_ACCESS,
        aws_secret_access_key=S3_SECRET,
        region_name="us-east-1"
    )
    all_objects = []
    # Paginator is used to list if more 1000 files
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    for page in page_iterator:
        for obj in page.get('Contents', []):
            key=obj['Key']
            last_modified = obj['LastModified']
            if last_modified > datetime.now(timezone.utc)- timedelta(days=days_lookback):
                 all_objects.append(obj)
    all_objects.sort(key=lambda x: x['LastModified'], reverse=True)
    #latest_files = all_objects[:limit]
    latest_files = all_objects
    paths = [f"s3a://{bucket_name}/{obj['Key']}" for obj in latest_files]
    return paths


In [15]:
latest_files = get_latest_file(
        bucket_name="trading-okx",
        prefix="bronze/okx_trades/",
        days_lookback=6
    )
latest_files

['s3a://trading-okx/bronze/okx_trades/2025-12-03/14/okx_trades_20251203_143941.jsonl.gz',
 's3a://trading-okx/bronze/okx_trades/2025-12-03/14/okx_trades_20251203_143932.jsonl.gz',
 's3a://trading-okx/bronze/okx_trades/2025-12-03/14/okx_trades_20251203_143931.jsonl.gz',
 's3a://trading-okx/bronze/okx_trades/2025-12-03/14/okx_trades_20251203_143930.jsonl.gz',
 's3a://trading-okx/bronze/okx_trades/2025-12-03/14/okx_trades_20251203_143929.jsonl.gz',
 's3a://trading-okx/bronze/okx_trades/2025-12-03/14/okx_trades_20251203_143928.jsonl.gz']

In [16]:
spark = SparkSession.builder \
        .appName("OKX_Bronze_To_Silver") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
        .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) \
        .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS) \
        .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .getOrCreate()


25/12/03 22:23:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [18]:
spark.stop()

In [17]:
trade_schema = StructType([
    StructField("instId", StringType(), True),
    StructField("tradeId", StringType(), True),
    StructField("px", StringType(), True),
    StructField("sz", StringType(), True),
    StructField("side", StringType(), True),
    StructField("ts", StringType(), True),
    StructField("count", StringType(), True),
    StructField("source", StringType(), True),
    StructField("seqId", StringType(), True)
])
schema = StructType([
    StructField("received_at", StringType(), True),
    StructField("payload", StructType([
        StructField("data", ArrayType(trade_schema), True)
    ]), True)
])

df = spark.read.schema(schema).json(latest_files)
df.show()

25/12/03 22:23:23 ERROR Executor: Exception in task 0.0 in stage 8.0 (TID 24)
org.apache.spark.SparkException: Encountered error while reading file s3a://trading-okx/bronze/okx_trades/2025-12-03/14/okx_trades_20251203_143941.jsonl.gz. Details:
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:863)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:293)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	

Py4JJavaError: An error occurred while calling o390.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 24) (10.255.255.254 executor driver): org.apache.spark.SparkException: Encountered error while reading file s3a://trading-okx/bronze/okx_trades/2025-12-03/14/okx_trades_20251203_143941.jsonl.gz. Details:
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:863)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:293)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIt

In [ ]:
df_exploded = df.select(
    col("received_at"),
    explode(col("payload.data")).alias("trade")
)
df_silver = df_exploded.select(
    col("trade.instId").alias("symbol"),
    col("trade.tradeId").alias("tradeId"),
    col("trade.side").alias("side"),
    col("trade.px").cast("double").alias("price"),
    col("trade.sz").cast("double").alias("quantity"),
    (col("trade.ts").cast("long") / 1000).cast("timestamp").alias("trade_time"),
    col("received_at").cast("timestamp").alias("ingestion_time")
)
df_silver.show()

In [ ]:


df_cleaned = df_silver \
    .dropna(subset=["price", "quantity"]) \
    .dropDuplicates(["tradeId"]) \
    .withColumn("date_part", date_format(col("trade_time"), "yyyy-MM-dd"))
df_cleaned.show()


In [ ]:
# spark connect
# should change config in .sh



#df.show(truncate=False)
 # Cột dùng để partition
#df_silver.show(truncate=False)

duck_path = '/mnt/d/learn/DE/Semina_project/datawarehouse.duckdb'
con = duckdb.connect(duck_path)
# read file sql to connect to minio and create dim fact table

# đoạn code dưới được đưa vào khi select from read_parquet(path), khi đọc file parquet trực tiếp vào duckdb thì
# run code nó sẽ set up các config cho minio để kết nối trực tiếp đến duckdb -> duckdb có thể read direct data in minio
# nếu ko có ko cần thêm vào
'''with open('/mnt/d/learn/DE/Semina_project/SQL_db/config_dw/warehouse_source.sql', 'r') as f:
     sql_script = f.read()
con.execute(sql_script)'''

# dki dataframe thành bảng ảo arrow_table_virtual để query sql
# convert pyspark dataframe to arrow table
arrow_table = pa.Table.from_pandas(df_silver.toPandas())
con.register("arrow_table_virtual", arrow_table)
con.execute('''
            INSERT INTO fact_trades(symbol,
                                    tradeId,
                                    side,
                                    price,
                                    quantity,
                                    trade_time,
                                    ingestion_time)
            SELECT symbol,
                   tradeId,
                   side,
                   price,
                   quantity,
                   trade_time,
                   ingestion_time
            FROM arrow_table_virtual
            ''')
#format data and load into minio with parquet
output = f"s3a://trading-okx/silver/okx_trades/"
df_silver.write.mode('append').format("parquet").save(output)


In [23]:
con.close()
spark.stop()
